<a href="https://colab.research.google.com/github/santoshrsarangi/tensorflow/blob/main/Pretrained_GloVe_Embedding_for_IMDB_Movie_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  10.0M      0  0:00:08  0:00:08 --:--:-- 16.9M


In [ ]:
!tar -xf aclImdb_v1.tar.gz

In [ ]:
!ls -l

total 82164
drwxr-xr-x 4 7297 1000     4096 Jun 26  2011 aclImdb
-rw-r--r-- 1 root root 84125825 Dec 29 05:10 aclImdb_v1.tar.gz
drwxr-xr-x 1 root root     4096 Dec 20 20:19 sample_data


In [ ]:
!rm -r aclImdb/train/unsup

In [ ]:
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

In [ ]:
import os, pathlib, shutil, random

In [ ]:
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

In [ ]:
for category in ("pos", "neg"):
  os.makedirs(val_dir / category)
  files = os.listdir(train_dir / category)
  random.Random(1337).shuffle(files)
  num_of_val_files = int(len(files) * 0.2)

  val_files = files[-num_of_val_files:]

  for fname in val_files:
    shutil.move(train_dir / category / fname, val_dir / category / fname)

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
batch_size = 32

In [ ]:
train_ds = keras.utils.text_dataset_from_directory("aclImdb/train", batch_size = batch_size)

Found 20000 files belonging to 2 classes.


In [ ]:
val_ds = keras.utils.text_dataset_from_directory("aclImdb/val", batch_size=batch_size)

Found 5000 files belonging to 2 classes.


In [ ]:
test_ds = keras.utils.text_dataset_from_directory("aclImdb/test", batch_size = batch_size)

Found 25000 files belonging to 2 classes.


In [ ]:
for inputs, targets in train_ds:
  print(inputs[0])
  print(targets[0])
  print(inputs.shape)
  print(targets.shape)
  print(inputs.dtype)
  print(targets.dtype)
  break

tf.Tensor(b"The real story (took place in Kansas in 1959) of a murder (Perry and Dick, two ex-convicts who broke into a remote house on a rainy night to steal and kill everyone they met). Richard Brooks directed the chilling and disturbing Capote's book about the reasons that drove these kids to the crime (Are they Natural Born Killers ?). The crime scenes are very brutal and haunting because of the lack of senses and reasons for what we witnessed. Stunning black & white cinematography from Conrand Hall, excellent country - road music score from Quincy Jones, amazing performances in two principal roles from Robert Blake and Scott Wilson and first time in a movie a sad comment about capital punishment at the last moments before their deaths. Jones, Hall and Brooks (as director and as writer for adapted screenplay) are Academy Award nominees. Gripping, superbly directed and frightening, one of the best films of this decade", shape=(), dtype=string)
tf.Tensor(1, shape=(), dtype=int32)
(32

In [ ]:
from keras.layers import TextVectorization

In [ ]:
max_length = 600
max_token = 20000

In [ ]:
text_vectorization = TextVectorization(output_mode="int", max_tokens=max_token, output_sequence_length=max_length)

In [ ]:
text_ds = train_ds.map(lambda x, y: x)

In [ ]:
text_vectorization.adapt(text_ds)

In [ ]:
ds = text_vectorization.get_vocabulary()

In [ ]:
print(ds)

['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i', 'this', 'that', 'br', 'was', 'as', 'for', 'with', 'but', 'movie', 'film', 'on', 'not', 'you', 'are', 'his', 'have', 'he', 'be', 'one', 'its', 'at', 'all', 'by', 'an', 'they', 'from', 'who', 'like', 'so', 'her', 'just', 'or', 'about', 'has', 'if', 'out', 'some', 'there', 'what', 'good', 'when', 'more', 'very', 'even', 'she', 'my', 'no', 'up', 'would', 'which', 'only', 'time', 'really', 'story', 'their', 'were', 'see', 'had', 'can', 'me', 'than', 'we', 'much', 'well', 'been', 'get', 'will', 'also', 'into', 'do', 'other', 'because', 'people', 'bad', 'first', 'great', 'how', 'most', 'him', 'dont', 'then', 'made', 'movies', 'make', 'any', 'way', 'films', 'could', 'them', 'after', 'too', 'characters', 'think', 'watch', 'two', 'many', 'character', 'being', 'seen', 'never', 'little', 'where', 'love', 'acting', 'plot', 'best', 'did', 'know', 'show', 'life', 'does', 'ever', 'your', 'better', 'over', 'still', 'off', 'end', 'these

In [ ]:
len(ds)

20000

In [ ]:
binary_1gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

In [ ]:
binary_1gram_val_ds = val_ds.map(lambda x, y : (text_vectorization(x), y), num_parallel_calls=4)

In [ ]:
binary_1gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

In [ ]:
for inputs, targets in binary_1gram_train_ds:
  print(inputs[0])
  print(targets[0])
  break

tf.Tensor(
[  768   679     5     1    11   394  3585  1162 15449     5  1629  5198
    14  1006   104    11     3   362    70    45  6262   985  7599   153
   188  3662   158   244    25     2   761 16411     1  1845  3039     2
  4067   306  1056     1    34   315    12   235     1   132  1407     2
  1700  4858     8    11    20   434     5 10654 19353    77   160    11
    20     6    28     4  2550    17  4422  2801     5  1632 17338    33
     2  1533   186    99   110    44     2   399  7193     6  4513    75
    39  3740     2   183    12    11    14    92    51     9    14   261
    22    61     2    77     5     2  3755    18     5     2   863   523
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0    

In [ ]:
def get_model(max_tokens=20000, hidden_dims=16):

  inputs = keras.Input(shape=(None,), dtype="int64")
  embeded = keras.layers.Embedding(input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs)

  x = keras.layers.Bidirectional(keras.layers.LSTM(32))(embeded)
  x = keras.layers.Dropout(0.5)(x)
  outputs = keras.layers.Dense(1, activation="sigmoid")(x)

  model = keras.Model(inputs, outputs)
  model.compile(optimizer="rmsprop",
                loss="binary_crossentropy",
                metrics=["accuracy"])
  
  return model


In [ ]:
model = get_model()

In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_1 (Embedding)     (None, None, 256)         5120000   
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               73984     
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5,194,049
Trainable params: 5,194,049
Non-trainable params: 0
_________________________________________________

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("embedding_layer_with_masked_zeros.keras", save_best_only=True)
]

In [ ]:
history = model.fit(binary_1gram_train_ds.cache(), 
                    validation_data = binary_1gram_val_ds.cache(),
                    callbacks=callbacks,
                    epochs=10)

Epoch 1/10
625/625 [==============================] - 45s 56ms/step - loss: 0.4826 - accuracy: 0.7806 - val_loss: 0.3898 - val_accuracy: 0.8514
Epoch 2/10
625/625 [==============================] - 28s 45ms/step - loss: 0.3169 - accuracy: 0.8849 - val_loss: 0.3368 - val_accuracy: 0.8834
Epoch 3/10
625/625 [==============================] - 28s 45ms/step - loss: 0.2423 - accuracy: 0.9167 - val_loss: 0.3600 - val_accuracy: 0.8744
Epoch 4/10
625/625 [==============================] - 29s 46ms/step - loss: 0.2014 - accuracy: 0.9309 - val_loss: 0.3083 - val_accuracy: 0.8846
Epoch 5/10
625/625 [==============================] - 29s 46ms/step - loss: 0.1733 - accuracy: 0.9424 - val_loss: 0.3306 - val_accuracy: 0.8868
Epoch 6/10
625/625 [==============================] - 31s 49ms/step - loss: 0.1554 - accuracy: 0.9489 - val_loss: 0.3179 - val_accuracy: 0.8816
Epoch 7/10
625/625 [==============================] - 29s 46ms/step - loss: 0.1264 - accuracy: 0.9599 - val_loss: 0.3799 - val_accuracy:

In [ ]:
history_masked_zeros = model.fit(binary_1gram_train_ds.cache(), 
                    validation_data = binary_1gram_val_ds.cache(),
                    callbacks=callbacks,
                    epochs=10)

Epoch 1/10
625/625 [==============================] - 48s 64ms/step - loss: 0.4000 - accuracy: 0.8191 - val_loss: 0.3337 - val_accuracy: 0.8532
Epoch 2/10
625/625 [==============================] - 31s 50ms/step - loss: 0.2258 - accuracy: 0.9125 - val_loss: 0.3172 - val_accuracy: 0.8672
Epoch 3/10
625/625 [==============================] - 31s 50ms/step - loss: 0.1620 - accuracy: 0.9398 - val_loss: 0.3116 - val_accuracy: 0.8710
Epoch 4/10
625/625 [==============================] - 33s 53ms/step - loss: 0.1237 - accuracy: 0.9572 - val_loss: 0.3220 - val_accuracy: 0.8714
Epoch 5/10
625/625 [==============================] - 31s 49ms/step - loss: 0.0924 - accuracy: 0.9682 - val_loss: 0.3644 - val_accuracy: 0.8752
Epoch 6/10
625/625 [==============================] - 31s 49ms/step - loss: 0.0680 - accuracy: 0.9777 - val_loss: 0.3620 - val_accuracy: 0.8774
Epoch 7/10
625/625 [==============================] - 31s 50ms/step - loss: 0.0510 - accuracy: 0.9833 - val_loss: 0.4192 - val_accuracy:

In [ ]:
#result
model.evaluate(binary_1gram_test_ds)

782/782 [==============================] - 22s 28ms/step - loss: 0.5836 - accuracy: 0.8372


[0.5835949182510376, 0.8371999859809875]

In [ ]:
#result with masked zeros
model.evaluate(binary_1gram_test_ds)

782/782 [==============================] - 21s 27ms/step - loss: 0.6054 - accuracy: 0.8490


[0.605360209941864, 0.8489599823951721]